**Importing the dependencies**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Device configuration


In [2]:
# check for CUDA availability
device = torch.device('cuda'if torch.cuda.is_available() else "cpu")
print(f"Using device:{device}")

Using device:cuda


Data Collection and preprocessing


In [3]:
data= load_breast_cancer()
x= data.data
y= data.target
print(x)
print(y)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 

In [4]:
print(y[:5])

[0 0 0 0 0]


In [5]:
# split dataset into training and testing set
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
print(x.shape, x_train.shape, x_test.shape)

(569, 30) (455, 30) (114, 30)


In [7]:
# standardize the data using standard scalar
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [8]:
type(x_train)

numpy.ndarray

In [9]:
# convert data to PyTorch tensors and move it to GPU
x_train=torch.tensor(x_train, dtype=torch.float32).to(device)
x_test=torch.tensor(x_test, dtype=torch.float32).to(device)
y_train=torch.tensor(y_train, dtype=torch.float32).to(device)
y_test=torch.tensor(y_test, dtype=torch.float32).to(device)

In [10]:
type(x_train)
type(y_test)

torch.Tensor

**Neural Network Architecture**

In [11]:
#define the neural network architecture
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, output_size): #input size= how many neurons should be in input layers
    super(NeuralNet, self).__init__()
    self.fc1=nn.Linear(input_size, hidden_size) # fc=fully connected layer
    self.relu=nn.ReLU() # to capture complex relationships in data
    self.fc2=nn.Linear(hidden_size, output_size)
    self.sigmoid=nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out



In [12]:
# define hyperparMETERS
input_size = x_train.shape[1]
hidden_size = 64
output_size = 1
num_epochs = 100
learning_rate = 0.001

In [13]:
# initialize the neural network and move it the GPU
model = NeuralNet(input_size, hidden_size, output_size).to(device)


In [14]:
#define loss and the optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# training the model
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(x_train)
  loss = criterion(outputs, y_train.view(-1,1))
  loss.backward()
  optimizer.step()

  # calculate accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum()
    accuracy = correct/y_train.size(0)

  if (epoch+1) % 10 == 0:
    print(f"Epoch[{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")


Epoch[10/100], Loss: 0.5325, Accuracy: 88.13%
Epoch[20/100], Loss: 0.4195, Accuracy: 91.43%
Epoch[30/100], Loss: 0.3349, Accuracy: 93.19%
Epoch[40/100], Loss: 0.2710, Accuracy: 94.95%
Epoch[50/100], Loss: 0.2236, Accuracy: 95.60%
Epoch[60/100], Loss: 0.1889, Accuracy: 96.70%
Epoch[70/100], Loss: 0.1634, Accuracy: 96.70%
Epoch[80/100], Loss: 0.1441, Accuracy: 96.92%
Epoch[90/100], Loss: 0.1291, Accuracy: 97.14%
Epoch[100/100], Loss: 0.1174, Accuracy: 97.36%


Model Evaluation

In [16]:
with torch.no_grad():
    model.eval()

    # Evaluation on training set
    train_outputs = model(x_train)
    train_predicted = train_outputs.round()
    train_accuracy = accuracy_score(y_train.cpu().numpy(), train_predicted.cpu().numpy())

    # Evaluation on test set
    test_outputs = model(x_test)
    test_predicted = test_outputs.round()
    test_accuracy = accuracy_score(y_test.cpu().numpy(), test_predicted.cpu().numpy())

    print(f"\nTraining Set Accuracy: {train_accuracy * 100:.2f}%")
    print(f"Test Set Accuracy: {test_accuracy * 100:.2f}%")


Training Set Accuracy: 97.36%
Test Set Accuracy: 96.49%


In [17]:
# evaluation on training set
model.eval()
with torch.no_grad():
  outputs=model(x_train)
  predicted=outputs.round()
  correct=(predicted==y_train.view(-1,1)).float().sum()
  accuracy=correct/y_train.size(0)
  print(f"Accuracy on training data: {accuracy.item()*100:.2f}%")

Accuracy on training data: 97.36%


In [21]:
# evaluation on test set
model.eval()
with torch.no_grad():
  outputs=model(x_test)
  predicted=outputs.round()
  correct=(predicted==y_test.view(-1,1)).float().sum()
  accuracy=correct/y_test.size(0)
  print(f"Accuracy on test data: {accuracy.item()*100:.2f}%")

Accuracy on test data: 96.49%
